In [14]:
import os
from dotenv import load_dotenv
from anydata import DataAPI
from datetime import datetime

In [2]:
load_dotenv()

True

### Stats Canada API

In [3]:
statcan_api = DataAPI("https://www150.statcan.gc.ca/t1/wds/rest/")

In [4]:
statcan_api.add_endpoint("getAllCubesList", method="GET")

In [5]:
statcan_api["getAllCubesList"].to_pandas()

,productId,cansimId,cubeTitleEn,cubeTitleFr,cubeStartDate,cubeEndDate,releaseTime,archived,subjectCode,surveyCode,frequencyCode,corrections,dimensions
0,10100001,183-0021,Federal public sector employment reconciliatio...,Emploi du secteur public fédéral rapprochement...,1999-01-01,2011-01-01,2012-08-01T12:30,1,[10],[1713],12,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
1,10100002,191-0002,Central government debt,Dette du gouvernement central,2009-04-01,2024-02-01,2024-04-17T12:30,2,[10],[7514],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
2,10100003,176-0079,Government of Canada debt securities: Gross ne...,Titres d’emprunt du gouvernement du Canada : é...,1975-01-01,2024-03-01,2024-04-24T12:30,2,[10],[7502],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
3,10100004,176-0013,"Chartered banks, total claims and liabilities ...","Banques à charte, ensembles des créances et en...",1978-04-01,2023-10-01,2024-03-21T12:30,2,[10],[7502],9,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
4,10100005,385-0041,Canadian Classification of Functions of Govern...,Classification canadienne des fonctions des ad...,2008-01-01,2022-01-01,2023-11-28T13:30,2,[10],[5218],12,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7394,98100644,None,Population who worked mainly full-time for mos...,Population ayant travaillé principalement à te...,2021-01-01,2021-01-01,2024-03-26T12:30,2,None,[3901],18,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
7395,98100645,None,"Class of worker by visible minority, selected ...",Catégorie de travailleur selon la minorité vis...,2021-01-01,2021-01-01,2024-03-26T12:30,2,None,[3901],18,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
7396,98100646,None,Legislative and senior management occupations ...,Membres des corps législatifs et cadres supéri...,2021-01-01,2021-01-01,2024-03-26T12:30,2,None,[3901],18,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
7397,98100648,None,"Youth not in education, employment or training...","Jeunes ni aux études, ni en emploi, ni en form...",2021-01-01,2021-01-01,2024-03-26T12:30,2,None,[3901],18,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."


In [15]:
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
timestamp_date_str = datetime.now().strftime("%Y%m%d")
timestamp_datetime_str = datetime.now().strftime("%Y%m%d%H%M%S")

if not os.path.exists(f"../data/raw/{timestamp_date_str}"):
    os.makedirs(f"../data/raw/{timestamp_date_str}")

response = statcan_api["getAllCubesList"].request()
if response.status_code == 200:
    with open(f"../data/raw/{timestamp_date_str}/all_cubes_{timestamp_datetime_str}.json", "w+") as f:
        f.write(response.text)

### DuckDB

In [12]:
import duckdb

In [17]:
# Create a new DuckDB database if not exists
sql_query = """SHOW TABLES
"""

with duckdb.connect("../data/duckdb_sandbox.db") as con:
    con.execute(sql_query).df()

In [19]:
# Create table and populate with json blob if not exists
sql_query_create_all_cubes = f"""CREATE OR REPLACE TABLE all_cubes AS
SELECT *
FROM read_json_auto('../data/raw/{timestamp_date_str}/all_cubes_{timestamp_datetime_str}.json')
"""

with duckdb.connect("../data/duckdb_sandbox.db") as con:
    con.sql(sql_query_create_all_cubes)

In [22]:
sql_query = """FROM all_cubes
"""

with duckdb.connect("../data/duckdb_sandbox.db") as con:
    display(con.execute(sql_query).df())

,productId,cansimId,cubeTitleEn,cubeTitleFr,cubeStartDate,cubeEndDate,releaseTime,archived,subjectCode,surveyCode,frequencyCode,corrections,dimensions
0,10100001,183-0021,Federal public sector employment reconciliatio...,Emploi du secteur public fédéral rapprochement...,1999-01-01,2011-01-01,2012-08-01T12:30,1,[10],[1713],12,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
1,10100002,191-0002,Central government debt,Dette du gouvernement central,2009-04-01,2024-02-01,2024-04-17T12:30,2,[10],[7514],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
2,10100003,176-0079,Government of Canada debt securities: Gross ne...,Titres d’emprunt du gouvernement du Canada : é...,1975-01-01,2024-03-01,2024-04-24T12:30,2,[10],[7502],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
3,10100004,176-0013,"Chartered banks, total claims and liabilities ...","Banques à charte, ensembles des créances et en...",1978-04-01,2023-10-01,2024-03-21T12:30,2,[10],[7502],9,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
4,10100005,385-0041,Canadian Classification of Functions of Govern...,Classification canadienne des fonctions des ad...,2008-01-01,2022-01-01,2023-11-28T13:30,2,[10],[5218],12,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7394,98100644,None,Population who worked mainly full-time for mos...,Population ayant travaillé principalement à te...,2021-01-01,2021-01-01,2024-03-26T12:30,2,NaN,[3901],18,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
7395,98100645,None,"Class of worker by visible minority, selected ...",Catégorie de travailleur selon la minorité vis...,2021-01-01,2021-01-01,2024-03-26T12:30,2,NaN,[3901],18,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
7396,98100646,None,Legislative and senior management occupations ...,Membres des corps législatifs et cadres supéri...,2021-01-01,2021-01-01,2024-03-26T12:30,2,NaN,[3901],18,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
7397,98100648,None,"Youth not in education, employment or training...","Jeunes ni aux études, ni en emploi, ni en form...",2021-01-01,2021-01-01,2024-03-26T12:30,2,NaN,[3901],18,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."


In [24]:
sql_query = """SELECT
    cubeTitleEn, cubeStartDate, cubeEndDate, releaseTime
  FROM all_cubes
"""

with duckdb.connect("../data/duckdb_sandbox.db") as con:
    display(con.execute(sql_query).df().sample(10))

,cubeTitleEn,cubeStartDate,cubeEndDate,releaseTime
1026,"Pap smear, by age group, women aged 18 to 69 y...",1994-01-01,1998-01-01,2017-02-27T13:30
2467,"Periodical publishers, summary statistics",2013-01-01,2021-01-01,2023-01-24T13:30
6348,Water intake in mineral extraction and thermal...,2005-01-01,2021-01-01,2024-03-18T12:30
6113,"Postsecondary graduates, by status of student ...",1992-01-01,2021-01-01,2023-11-22T13:30
2435,"Consulting services, summary statistics, inactive",1998-01-01,2012-01-01,2017-02-06T13:30
6440,Resource-based community status and proportion...,2021-01-01,2021-01-01,2023-12-13T13:30
7042,First official language spoken by language spo...,2021-01-01,2021-01-01,2022-08-17T12:30
5777,"International investment position, Canada's gr...",2002-10-01,2023-10-01,2024-03-12T12:30
6083,Trade and vocational training education expend...,1954-01-01,2004-01-01,2010-02-11T13:30
3288,"Innovation and business strategy, percentage o...",2009-01-01,2012-01-01,2014-06-17T12:30


In [35]:
sql_query = """
  FROM all_cubes
  WHERE archived = 2 AND cubeEndDate > '2024-01-01'
"""

with duckdb.connect("../data/duckdb_sandbox.db") as con:
    display(con.execute(sql_query).df())

,productId,cansimId,cubeTitleEn,cubeTitleFr,cubeStartDate,cubeEndDate,releaseTime,archived,subjectCode,surveyCode,frequencyCode,corrections,dimensions
0,10100002,191-0002,Central government debt,Dette du gouvernement central,2009-04-01,2024-02-01,2024-04-17T12:30,2,[10],[7514],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
1,10100003,176-0079,Government of Canada debt securities: Gross ne...,Titres d’emprunt du gouvernement du Canada : é...,1975-01-01,2024-03-01,2024-04-24T12:30,2,[10],[7502],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
2,10100006,176-0091,"Funds advanced, outstanding balances, and inte...","Avances de fonds, encours des prêts, et taux d...",2013-01-01,2024-02-01,2024-04-22T12:30,2,[10],[7502],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
3,10100106,176-0003,"Consumer Price Index (CPI) statistics, alterna...",Statistiques de l'Indice des prix à la consomm...,1979-11-01,2024-03-01,2024-04-17T12:30,2,[10],[7502],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
4,10100108,176-0010,"Bank of Canada, assets and liabilities, at mon...","Banque du Canada, actif et passif, en fin de mois",1935-01-01,2024-03-01,2024-04-11T12:30,2,[10],[7502],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,36100640,None,Credit liabilities of private non-financial co...,Passifs de crédits des sociétés non-financière...,1990-01-01,2024-02-01,2024-04-18T12:30,2,[36],[1806],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
266,36100641,None,Credit assets of the financial corporations se...,Actifs de crédits du secteur des sociétés fina...,1990-01-01,2024-02-01,2024-04-18T12:30,2,[36],[1806],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
267,36100642,None,New issuances of securities by type of use of ...,Nouvelles émissions de titres financiers selon...,2020-01-01,2024-02-01,2024-04-17T12:30,2,[36],[5285],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."
268,36100666,None,"Selected credit aggregates, monthly","Estimations de crédit sélectionnées, mensuelles",1990-01-01,2024-02-01,2024-04-18T12:30,2,[36],[1806],6,[],"[{'dimensionNameEn': 'Geography', 'dimensionNa..."


In [32]:
sql_query = """
  FROM information_schema.columns
  WHERE column_name = 'cubeEndDate'
"""

with duckdb.connect("../data/duckdb_sandbox.db") as con:
    display(con.execute(sql_query).df())

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable,COLUMN_COMMENT
0,duckdb_sandbox,main,all_cubes,cubeEndDate,6,None,YES,DATE,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
